In [305]:
%load_ext autoreload
%autoreload 2
from sympy import symbols, init_printing, simplify, solve, srepr, Add, Symbol, Integer, Float, pi, cos, sin, Rational
from sympy import IndexedBase, Eq, simplify,sqrt
from grid import *
init_printing()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [306]:
Txx = Field('Txx', (False,False,False,False))
Tyy = Field('Tyy', (False,False,False,False))
Tzz = Field('Tzz', (False,False,False,False))
Txy = Field('Txy', (False,True,True,False))
Tyz = Field('Tyz', (False,False,True,True))
Txz = Field('Txz', (False,True,False,True))
U = Field('U', (True,True,False,False))
V = Field('V', (True,False,True,False))
W = Field('W', (True,False,False,True))

In [307]:
grid = StaggeredGrid3D('h h h','dt','dimx dimy dimz')
grid.set_stress_fields([Txx,Tyy,Tzz,Txy,Tyz,Txz])
grid.set_velocity_fields([U,V,W])
U.associate_stress_fields((Txx,Txy,Txz))
V.associate_stress_fields((Txy,Tyy,Tyz))
W.associate_stress_fields((Txz,Tyz,Tzz))

####Analytical solutions
$U = cos(\pi x)(sin(\pi y)-sin(\pi z))cos(\Omega t)$<br>
$V = cos(\pi y)(sin(\pi z)-sin(\pi x))cos(\Omega t)$<br>
$W = cos(\pi z)(sin(\pi x)-sin(\pi y))cos(\Omega t)$<br>
$T_{xx} = -Asin(\pi x)(sin(\pi y)-sin(\pi z))sin(\Omega t)$<br>
$T_{yy} = -Asin(\pi y)(sin(\pi z)-sin(\pi x))sin(\Omega t)$<br>
$T_{zz} = -Asin(\pi z)(sin(\pi x)-sin(\pi y))sin(\Omega t)$<br>
$T_{xy} = T_{yz} = T_{xz} = 0$<br>
where $A=\sqrt{2\rho \mu} \space \space b=\pi\sqrt{\frac{2\mu}{\rho}}$

In [308]:
rho, beta, lam, mu, h, dt= symbols('rho beta lambda mu h dt')
x,y,z,t = symbols('x y z t')
Omega = pi*sqrt(2*mu/rho)
A = sqrt(2*rho*mu)
U_func = cos(pi*x)*(sin(pi*y)-sin(pi*z))*cos(Omega*t)
V_func = cos(pi*y)*(sin(pi*z)-sin(pi*x))*cos(Omega*t)
W_func = cos(pi*z)*(sin(pi*x)-sin(pi*y))*cos(Omega*t)
Txx_func = -A*sin(pi*x)*(sin(pi*y)-sin(pi*z))*sin(Omega*t)
Tyy_func = -A*sin(pi*y)*(sin(pi*z)-sin(pi*x))*sin(Omega*t)
Tzz_func = -A*sin(pi*z)*(sin(pi*x)-sin(pi*y))*sin(Omega*t)
Txy_func = Float(0)
Tyz_func = Float(0)
Txz_func = Float(0)

In [309]:
U.set_analytic_func(U_func)
V.set_analytic_func(V_func)
W.set_analytic_func(W_func)
Txx.set_analytic_func(Txx_func)
Tyy.set_analytic_func(Tyy_func)
Tzz.set_analytic_func(Tzz_func)
Txy.set_analytic_func(Txy_func)
Tyz.set_analytic_func(Tyz_func)
Txz.set_analytic_func(Txz_func)

In [310]:
grid.calc_derivatives()

###PDEs
#####momentum equations
$\partial_tU = b(\partial_xT_{xx}+\partial_yT_{xy}+\partial_zT_{xz})$<br>
$\partial_tV = b(\partial_xT_{xy}+\partial_yT_{yy}+\partial_zT_{yz})$<br>
$\partial_tW = b(\partial_xT_{xz}+\partial_yT_{yz}+\partial_zT_{zz})$<br>
#####stress-strain equations
$\partial_tT_{xx} = (\lambda+2\mu)\partial_xU + \lambda(\partial_yV+\partial_zW)$<br>
$\partial_tT_{yy} = (\lambda+2\mu)\partial_yV + \lambda(\partial_xU+\partial_zW)$<br>
$\partial_tT_{zz} = (\lambda+2\mu)\partial_zW + \lambda(\partial_xU+\partial_yV)$<br>
$\partial_tT_{xy} = \mu(\partial_yU + \partial_xV)$<br>
$\partial_tT_{xz} = \mu(\partial_zU + \partial_xW)$<br>
$\partial_tT_{yz} = \mu(\partial_zV + \partial_yW)$<br>

In [311]:
# momentum equations
eq1 = Eq(U.d[0][1], beta*(Txx.d[1][2] + Txy.d[2][2] + Txz.d[3][2]))
eq2 = Eq(V.d[0][1], beta*(Txy.d[1][2] + Tyy.d[2][2] + Tyz.d[3][2]))
eq3 = Eq(W.d[0][1], beta*(Txz.d[1][2] + Tyz.d[2][2] + Tzz.d[3][2]))
# stress-strain equations
eq4 = Eq(Txx.d[0][1], (lam + 2*mu)*U.d[1][2] + lam*(V.d[2][2]+W.d[3][2]))
eq5 = Eq(Tyy.d[0][1], (lam + 2*mu)*V.d[2][2] + lam*(U.d[1][2]+W.d[3][2]))
eq6 = Eq(Tzz.d[0][1], (lam + 2*mu)*W.d[3][2] + lam*(U.d[1][2]+V.d[2][2]))
eq7 = Eq(Txy.d[0][1], mu*(U.d[2][2] + V.d[1][2]))
eq8 = Eq(Tyz.d[0][1], mu*(V.d[3][2] + W.d[2][2]))
eq9 = Eq(Txz.d[0][1], mu*(U.d[3][2] + W.d[1][2]))

In [312]:
grid.solve_fd([eq1,eq2,eq3,eq4,eq5,eq6,eq7,eq8,eq9])

In [313]:
grid.velocity_loop()

u'#pragma omp for\r\nfor(int x=2;x<h - 2;++x){\r\n    for(int y=2;y<h - 2;++y){\r\n    \tfor(int z=2;y<h - 2;++z){\r\n    \t\tU[t1][x][y][z]=(1.0F/24.0F)*(-27*beta*dt*Txx[t1][x][y][z] + beta*dt*Txx[t1][x - 1][y][z] + 27*beta*dt*Txx[t1][x + 1][y][z] - beta*dt*Txx[t1][x + 2][y][z] + 27*beta*dt*Txy[t1][x][y][z] + beta*dt*Txy[t1][x][y - 2][z] - 27*beta*dt*Txy[t1][x][y - 1][z] - beta*dt*Txy[t1][x][y + 1][z] + 27*beta*dt*Txz[t1][x][y][z] + beta*dt*Txz[t1][x][y][z - 2] - 27*beta*dt*Txz[t1][x][y][z - 1] - beta*dt*Txz[t1][x][y][z + 1] + 24*h*U[t][x][y][z])/h;\n\t\t\tV[t1][x][y][z]=(1.0F/24.0F)*(27*beta*dt*Txy[t1][x][y][z] + beta*dt*Txy[t1][x - 2][y][z] - 27*beta*dt*Txy[t1][x - 1][y][z] - beta*dt*Txy[t1][x + 1][y][z] - 27*beta*dt*Tyy[t1][x][y][z] + beta*dt*Tyy[t1][x][y - 1][z] + 27*beta*dt*Tyy[t1][x][y + 1][z] - beta*dt*Tyy[t1][x][y + 2][z] + 27*beta*dt*Tyz[t1][x][y][z] + beta*dt*Tyz[t1][x][y][z - 2] - 27*beta*dt*Tyz[t1][x][y][z - 1] - beta*dt*Tyz[t1][x][y][z + 1] + 24*h*V[t][x][y][z])/h;\n\t\t\

In [314]:
Txx_expr = (lam + 2*mu)*U.d[1][1] + lam*(V.d[2][1]+W.d[3][1]); Txx.set_dt(Txx_expr)
Tyy_expr = (lam + 2*mu)*V.d[2][1] + lam*(U.d[1][1]+W.d[3][1]); Tyy.set_dt(Tyy_expr)
Tzz_expr = (lam + 2*mu)*W.d[3][2] + lam*(U.d[1][2]+V.d[2][2]); Tzz.set_dt(Tzz_expr)
Txy_expr = mu*(U.d[2][1] + V.d[1][1]); Txy.set_dt(Txy_expr)
Tyz_expr = mu*(V.d[3][1] + W.d[2][1]); Tyz.set_dt(Tyz_expr)
Txz_expr = mu*(U.d[3][1] + W.d[1][1]); Txz.set_dt(Txz_expr)

In [315]:
Eq(Txy_expr.subs(x,1+hf),Txy_expr.subs(x,2+hf))

In [316]:
grid.set_free_surface_boundary(1,0);grid.set_free_surface_boundary(1,1)
grid.set_free_surface_boundary(2,0);grid.set_free_surface_boundary(2,1)
grid.set_free_surface_boundary(3,0);grid.set_free_surface_boundary(3,1)

In [318]:
f= open('test00.cpp','w')
f.write(grid.velocity_bc())
f.close()

In [ ]:
Txy.bc

In [ ]:
d = 1
[f for f in grid.vfields if not f.offset[d]] + [f for f in grid.vfields if f.offset[d]]

In [ ]:
U.name[l]

In [ ]:
grid.stress_bc()